In [73]:
import pandas as pd
import numpy as np
from json import loads, dumps

In [87]:
table = pd.read_csv("/Users/farhadghassemi/Library/CloudStorage/OneDrive-Personal/Courses/Stanford/Conversational Virtual Assistant with Deep Learning/Project/Results/result_wikipedia_model_wikidata_compmix_dev_set_gpt_domain_not_passed_generic_examples_from_WWQ_compmix_max_entities_4_extended_prompt.csv")
table.sort_values(by=['question_id'], inplace=True)

In [88]:
table.head()

,question_id,domain,question,gold_entity_id,gold_entity_label,predicted_entity_id,predicted_entity_label,predicted_entity_score
1510,4966,books,Which is the first book of the book series Div...,Q16514101,Divergent,NaN,NaN,NaN
1509,4966,books,Which is the first book of the book series Div...,NaN,NaN,Q627192,Divergent (novel),0.4737
1508,4966,books,Which is the first book of the book series Div...,NaN,NaN,Q328212,Veronica Roth,0.9756
2309,4967,books,Who is the author of the book Divergent (novel)?,NaN,NaN,Q328212,Veronica Roth,0.9756
2308,4967,books,Who is the author of the book Divergent (novel)?,Q627192,Divergent (novel),Q627192,Divergent (novel),0.4737


In [90]:
table['predicted_entities'] = table[['predicted_entity_label', 'predicted_entity_id']].values.tolist()
table['gold_entities'] = table[['gold_entity_label', 'gold_entity_id']].values.tolist()
table.drop(['gold_entity_id', 'gold_entity_label', 'predicted_entity_id', 'predicted_entity_label', 'predicted_entity_score'], axis=1, inplace=True)

In [91]:
table.head()

,question_id,domain,question,predicted_entities,gold_entities
1510,4966,books,Which is the first book of the book series Div...,"[nan, nan]","[Divergent, Q16514101]"
1509,4966,books,Which is the first book of the book series Div...,"[Divergent (novel), Q627192]","[nan, nan]"
1508,4966,books,Which is the first book of the book series Div...,"[Veronica Roth, Q328212]","[nan, nan]"
2309,4967,books,Who is the author of the book Divergent (novel)?,"[Veronica Roth, Q328212]","[nan, nan]"
2308,4967,books,Who is the author of the book Divergent (novel)?,"[Divergent (novel), Q627192]","[Divergent (novel), Q627192]"


In [92]:
table2 = table.groupby("question_id").agg({'domain': 'first', 'question': 'first', 'predicted_entities': lambda x: list(x), 'gold_entities': lambda x: list(x)})
table2['gold_entities'] = table2['gold_entities'].apply(lambda x: [xi for xi in x if not pd.isna(xi[0])])
table2['predicted_entities'] = table2['predicted_entities'].apply(lambda x: [xi for xi in x if not pd.isna(xi[0])])
table2['question_id'] = table2.index

In [93]:
table2.head()

,domain,question,predicted_entities,gold_entities,question_id
question_id,,,,,
4966,books,Which is the first book of the book series Div...,"[[Divergent (novel), Q627192], [Veronica Roth,...","[[Divergent, Q16514101]]",4966
4967,books,Who is the author of the book Divergent (novel)?,"[[Veronica Roth, Q328212], [Divergent (novel),...","[[Divergent (novel), Q627192]]",4967
4968,books,What is the date of birth of the Divergent's a...,"[[Veronica Roth, Q328212]]","[[Divergent, Q627192], [Veronica Roth, Q328212]]",4968
4969,books,Which is the second book of the series Divergent?,[],"[[Divergent, Q16514101]]",4969
4970,books,What is the publication date of the Insurgent ...,"[[Insurgent (novel), Q5918512]]","[[Insurgent (novel), Q5918512]]",4970


In [94]:
result = table2.to_json("/Users/farhadghassemi/Library/CloudStorage/OneDrive-Personal/Courses/Stanford/Conversational Virtual Assistant with Deep Learning/Project/Results/result_wikipedia_model_wikidata_compmix_dev_set_gpt_domain_not_passed_generic_examples_from_WWQ_compmix_max_entities_4_extended_prompt.json", orient="records", indent=3)